In [42]:
!pip install InfluxDB

In [43]:
## Getting the data

from influxdb import InfluxDBClient # install via "pip install influxdb"
import pandas as pd



In [44]:
client = InfluxDBClient(host='influxus.itu.dk', port=8086, username='lsda', password='icanonlyread')
client.switch_database('orkney')

def get_df(results):
    values = results.raw["series"][0]["values"]
    columns = results.raw["series"][0]["columns"]
    df = pd.DataFrame(values, columns=columns).set_index("time")
    df.index = pd.to_datetime(df.index) # Convert to datetime-index
    return df

# Get the last 90 days of power generation data
generation = client.query(
    "SELECT * FROM Generation where time > now()-90d"
    ) # Query written in InfluxQL

# Get the last 90 days of weather forecasts with the shortest lead time
wind  = client.query(
    "SELECT * FROM MetForecasts where time > now()-90d and time <= now() and Lead_hours = '1'"
    ) # Query written in InfluxQL

gen_df = get_df(generation)
wind_df = get_df(wind)


####################################################


In [45]:
print (gen_df)

                             ANM  Non-ANM  Total
time                                            
2020-07-09 16:08:00+00:00  0.342    0.887  1.229
2020-07-09 16:09:00+00:00  0.297    0.907  1.204
2020-07-09 16:10:00+00:00  0.227    0.927  1.154
2020-07-09 16:11:00+00:00  0.248    0.936  1.184
2020-07-09 16:12:00+00:00  0.317    0.894  1.211
...                          ...      ...    ...
2020-10-07 16:03:00+00:00  2.329    4.775  7.104
2020-10-07 16:04:00+00:00  2.602    5.202  7.804
2020-10-07 16:05:00+00:00  2.487    5.023  7.510
2020-10-07 16:06:00+00:00  2.441    5.023  7.464
2020-10-07 16:07:00+00:00  2.432    5.321  7.753

[129339 rows x 3 columns]


In [48]:
gen_df.resample('180min').mean()

,ANM,Non-ANM,Total
time,,,
2020-07-09 15:00:00+00:00,0.302562,0.980580,1.283143
2020-07-09 18:00:00+00:00,0.354389,0.798806,1.153194
2020-07-09 21:00:00+00:00,-0.006073,1.167754,1.161682
2020-07-10 00:00:00+00:00,0.135567,1.191305,1.326872
2020-07-10 03:00:00+00:00,0.685582,1.885825,2.571407
...,...,...,...
2020-10-07 03:00:00+00:00,3.142150,4.176633,7.318783
2020-10-07 06:00:00+00:00,3.621500,5.775356,9.396856
2020-10-07 09:00:00+00:00,2.221422,3.301906,5.523328


In [49]:
wind_df

,Direction,Lead_hours,Source_time,Speed
time,,,,
2020-07-09 18:00:00+00:00,NNW,1,1594306800,3.12928
2020-07-09 21:00:00+00:00,NW,1,1594317600,1.78816
2020-07-10 00:00:00+00:00,W,1,1594328400,3.12928
2020-07-10 06:00:00+00:00,WSW,1,1594350000,4.91744
2020-07-10 09:00:00+00:00,W,1,1594360800,4.91744
...,...,...,...,...
2020-10-07 03:00:00+00:00,NW,1,1602028800,4.02336
2020-10-07 06:00:00+00:00,NW,1,1602039600,8.94080
2020-10-07 09:00:00+00:00,NW,1,1602050400,7.15264


In [51]:
new_df = pd.merge(wind_df, gen_df, on=["time"])

In [52]:
new_df

,Direction,Lead_hours,Source_time,Speed,ANM,Non-ANM,Total
time,,,,,,,
2020-07-09 18:00:00+00:00,NNW,1,1594306800,3.12928,0.485000,1.019,1.504000
2020-07-09 21:00:00+00:00,NW,1,1594317600,1.78816,0.035000,0.635,0.670000
2020-07-10 00:00:00+00:00,W,1,1594328400,3.12928,0.027000,1.046,1.073000
2020-07-10 06:00:00+00:00,WSW,1,1594350000,4.91744,1.180000,1.782,2.962000
2020-07-10 09:00:00+00:00,W,1,1594360800,4.91744,2.431000,4.044,6.475000
...,...,...,...,...,...,...,...
2020-10-07 03:00:00+00:00,NW,1,1602028800,4.02336,1.937000,3.536,5.473000
2020-10-07 06:00:00+00:00,NW,1,1602039600,8.94080,4.071999,6.060,10.131999
2020-10-07 09:00:00+00:00,NW,1,1602050400,7.15264,2.929000,3.654,6.583000


In [ ]:
# from sklearn.preprocessing import StandardScaler
# import numpy as np

# #### try with min max scalar##########
# ##### You asked me to give graph for test and predict data ####
# # create an object of the StandardScaler
# scaler = StandardScaler()
# # fit with the Item_MRP
# scaler.fit(np.array(new_df.Speed).reshape(-1,1))
# # transform the data
# new_df.Total = scaler.transform(np.array(new_df.Total).reshape(-1,1))
# new_df.Total


time
2020-07-09 18:00:00+00:00   -1.152429
2020-07-09 21:00:00+00:00   -1.229000
2020-07-10 00:00:00+00:00   -1.192000
2020-07-10 06:00:00+00:00   -1.018567
2020-07-10 09:00:00+00:00   -0.696031
                               ...   
2020-10-07 03:00:00+00:00   -0.788026
2020-10-07 06:00:00+00:00   -0.360274
2020-10-07 09:00:00+00:00   -0.686115
2020-10-07 12:00:00+00:00   -0.925745
2020-10-07 15:00:00+00:00   -0.206856
Name: Total, Length: 707, dtype: float64

In [53]:
print(new_df.isna().sum())



Direction      0
Lead_hours     0
Source_time    0
Speed          0
ANM            0
Non-ANM        0
Total          0
dtype: int64


In [54]:
new_df

,Direction,Lead_hours,Source_time,Speed,ANM,Non-ANM,Total
time,,,,,,,
2020-07-09 18:00:00+00:00,NNW,1,1594306800,3.12928,0.485000,1.019,1.504000
2020-07-09 21:00:00+00:00,NW,1,1594317600,1.78816,0.035000,0.635,0.670000
2020-07-10 00:00:00+00:00,W,1,1594328400,3.12928,0.027000,1.046,1.073000
2020-07-10 06:00:00+00:00,WSW,1,1594350000,4.91744,1.180000,1.782,2.962000
2020-07-10 09:00:00+00:00,W,1,1594360800,4.91744,2.431000,4.044,6.475000
...,...,...,...,...,...,...,...
2020-10-07 03:00:00+00:00,NW,1,1602028800,4.02336,1.937000,3.536,5.473000
2020-10-07 06:00:00+00:00,NW,1,1602039600,8.94080,4.071999,6.060,10.131999
2020-10-07 09:00:00+00:00,NW,1,1602050400,7.15264,2.929000,3.654,6.583000


In [55]:
new_df = new_df.drop(columns=["ANM", "Non-ANM"])

In [56]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 707 entries, 2020-07-09 18:00:00+00:00 to 2020-10-07 15:00:00+00:00
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Direction    707 non-null    object 
 1   Lead_hours   707 non-null    object 
 2   Source_time  707 non-null    int64  
 3   Speed        707 non-null    float64
 4   Total        707 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 33.1+ KB


In [57]:
import pandas as pd
from sklearn.model_selection import train_test_split



# Separate target from predictors
y = new_df.Total
X = new_df.drop(['Total'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 20 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [58]:
categorical_cols


['Direction', 'Lead_hours']

In [59]:
numerical_cols

['Source_time', 'Speed']

In [60]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder # changing the string data to categorical data like changing Direction to numbers

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols),
    ])


In [73]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression


#model = RandomForestRegressor(n_estimators=100, random_state=15)
model = LinearRegression()
from sklearn.metrics import mean_absolute_error

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)



Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='constant',
                                                                verbose=0),
                                                  ['Source_time', 'Speed']),
                                                 ('cat',
                                                  Pipeline(memory=None,
            

In [74]:
# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 3.7902207300843775


In [63]:
print(X_train.head())

                          Direction Lead_hours  Source_time     Speed
time                                                                 
2020-08-10 06:00:00+00:00        SE          1   1597028400   7.15264
2020-09-14 12:00:00+00:00        SW          1   1600074000  11.17600
2020-08-18 12:00:00+00:00        SE          1   1597741200   3.12928
2020-08-18 21:00:00+00:00        SE          1   1597773600   3.12928
2020-09-30 21:00:00+00:00       NNW          1   1601488800   8.94080


In [64]:

# Get all future forecasts regardless of lead time
forecasts  = client.query(
    "SELECT * FROM MetForecasts where time > now()"
    ) # Query written in InfluxQL
for_df = get_df(forecasts)

# Limit to only the newest source time
newest_source_time = for_df["Source_time"].max()
newest_forecasts = for_df.loc[for_df["Source_time"] == newest_source_time].copy()
print(newest_forecasts)

# Preprocess the forecasts and do predictions in one fell swoop 
# using your pipeline


                          Direction Lead_hours  Source_time     Speed
time                                                                 
2020-10-07 18:00:00+00:00       WNW          3   1602075600   3.12928
2020-10-07 21:00:00+00:00       WNW          6   1602075600   8.04672
2020-10-08 00:00:00+00:00       WNW          9   1602075600  11.17600
2020-10-08 03:00:00+00:00       WNW         12   1602075600   9.83488
2020-10-08 06:00:00+00:00        NW         15   1602075600   8.04672
2020-10-08 09:00:00+00:00       WNW         18   1602075600   7.15264
2020-10-08 12:00:00+00:00       WNW         21   1602075600   7.15264
2020-10-08 15:00:00+00:00         W         24   1602075600   8.04672
2020-10-08 18:00:00+00:00         W         27   1602075600   8.04672
2020-10-08 21:00:00+00:00         W         30   1602075600   8.04672
2020-10-09 00:00:00+00:00         W         33   1602075600   7.15264
2020-10-09 03:00:00+00:00         W         36   1602075600   4.91744
2020-10-09 06:00:00+

In [65]:
y=my_pipeline.predict(newest_forecasts)

In [66]:
y

array([ 3.93421464, 17.60430972, 26.30346113, 22.57525338, 14.69794767,
       15.11883789, 15.11883789, 15.71921519, 15.71921519, 15.71921519,
       13.23374336,  7.02006378,  4.15988206,  3.93421464,  6.41968648,
        5.99879626,  6.32963276, 12.54331234, 14.69794767, 17.1834195 ,
       23.39709908, 23.72793558, 26.21340741, 26.21340741, 28.69887925,
       26.21340741, 26.21340741, 23.72793558, 19.99972783, 15.02878417,
       12.54331234,  8.48426809,  8.90515831,  7.70590466])